In [1]:
import pandas as pd
import numpy as np
import librosa

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# load the resampled data

# this notebook is the demonstration of how we transform the resampled data into
# 4d numpy array which can be directly fit to Keras network for training

# the output dataframe we got from data_preparation script, after the resampling step
# this is a sample that contains only 100 instances for this demonstration

df = pd.read_pickle('gmd_100sample.pkl')
df[['audio_wav_resample', 'resample_sr', 'label']].head()

,audio_wav_resample,resample_sr,label
6906,"[0.00044701237, 0.00055765896, 0.00023127568, ...",39378,SD
273395,"[-0.003642959, -0.0011851551, 0.00021736528, -...",39378,SD
5982,"[-0.00014135065, 3.2329826e-05, 0.00019071152,...",39378,SD
310019,"[0.016627032, 0.019429555, 0.01696365, 0.01515...",39378,FT
246505,"[-0.029088601, -0.031084342, -0.031901143, -0....",137825,KD


In [3]:
# label encoding            
# some instances there are multiple positive labels

mlb = MultiLabelBinarizer()
y = pd.DataFrame(mlb.fit_transform(df['label']),columns=mlb.classes_)
y

,C,D,F,H,HH_close,HH_open,HT,K,KD,M,...,e,i,k,l,n,o,p,s,t,x
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,1,0,1,0,0
96,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
98,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# in the groove set there are 31 different labels

In [5]:
# select only the the most common labels to model for now  
# if we can find some solution(s) that work out well        then we expand the labels and data

cols = ['D','S','H','_','e','o','c','s','l','K','C']
label = y[cols]

# since we leave out some labels     certain instances will have "no label" no
# we encode these as "other"

# (this is one of the things that are uncertain choices, this is the usual way to do when we leave out some labels)
# we can also ...   
# 1. discard the no-label instances
# 2. just leave them as a bunch of zeros       [0,0,0..., 0]

# this will be tested and we just take what turns out to work best since each way have it's own logic

label['sum'] = label[cols].sum(axis = 1)

def other_label(label_sum):
    if label_sum == 0:
        return 1
    else: 
        return 0

label['other'] = label.apply(lambda row: other_label(row['sum']),axis=1)
label.drop(columns=['sum'],inplace=True)

label.head()

,D,S,H,_,e,o,c,s,l,K,C,other
0,1,1,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,1,0,0


In [6]:
# about 13.6% of the instances in the gmd set are labeled as "other",
# namely, non of the top 11 labels appeared in these instances

In [7]:
# turn the resampled autio_wav into the representation of short-time fourier transformation (stft)

stft_train = []

for i in range(df.shape[0]):
    stft_train.append(np.abs(librosa.stft(df.audio_wav_resample.iloc[i])))

X1 = np.array(stft_train)
X1 = X1.reshape(X1.shape[0],X1.shape[1],X1.shape[2],1)
X1.shape
    
# this is in the format of 
# a 4d array of (number of instances, y axis shape, x axis shape, 1 channel)

(100, 1025, 11, 1)

In [ ]:
# 100 stft representation of 100 instances, each has the shape (1025,11), has 1 channel

# (normal image classification tasks the instances will have multiple channels)
# the task we're doing is audio classification, which will only have 1 channel for all representation formats

In [8]:
# turn the resampled autio_wav into the representation of Mel-Frequency Spectrogram 

mel_train = []

for i in range(df.shape[0]):
    mel_train.append(librosa.feature.melspectrogram(y=df.audio_wav_resample.iloc[i], 
                                                    sr=df.resample_sr.iloc[i], 
                                                    n_mels=128, fmax=8000))

X2 = np.array(mel_train)
X2 = X2.reshape(X2.shape[0],X2.shape[1],X2.shape[2],1)
X2.shape

(100, 128, 11, 1)

In [9]:
# turn the resampled autio_wav into the representation of Mel-Frequency Cepstral Coefficients

mfcc_train = []

for i in range(df.shape[0]):
    mfcc_train.append(librosa.feature.mfcc(y=df.audio_wav_resample.iloc[i], 
                                           sr=df.resample_sr.iloc[i]         
                                           ))                            

X3 = np.array(mfcc_train)
X3 = X3.reshape(X3.shape[0],X3.shape[1],X3.shape[2],1)

X3.shape

(100, 20, 11, 1)

In [10]:
y = np.array(label)
x_train, x_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)

x_train.shape, x_test.shape

((80, 1025, 11, 1), (20, 1025, 11, 1))

In [11]:
y_train.shape, y_test.shape

((80, 12), (20, 12))

In [12]:
# can use np.save('gmd_stft.npy', X1) to save the ndarray data as npy files
# use np.load(filename) to load whenever we need it

np.save('gmd_stft_sample.npy',X1)

In [ ]:
# after finishing designing a network structure in Keras, data can be directly fit into the model like this for training

history = model.fit(x_train, y_train, 
                    validation_data=(x_test, y_test), 
                    validation_split=0.2, 
                    epochs=15, 
                    batch_size=64)